In [ ]:
#essa é a arte grafica 01

CÓDIGO FEITO PELO CHAT GPT 4.0

In [ ]:
import os
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import openpyxl
from PyPDF2 import PdfMerger

# Função para selecionar a pasta de PDFs
def selecionar_pasta():
    pasta = filedialog.askdirectory()
    if pasta:
        entrada_pasta.set(pasta)

# Função para selecionar o arquivo Excel
def selecionar_planilha():
    arquivo = filedialog.askopenfilename(filetypes=[("Excel files", "*.xlsx")])
    if arquivo:
        entrada_planilha.set(arquivo)

# Função para obter a lista de nomes a partir da planilha
def obter_lista_nomes(caminho_planilha):
    workbook = openpyxl.load_workbook(caminho_planilha)
    sheet = workbook.active
    # Começar a partir da linha 2 e ir de cima para baixo
    lista_nomes = [cell.value.strip() for cell in sheet['A'][1:] if cell.value is not None]
    workbook.close()
    return lista_nomes

# Função para renomear os arquivos na pasta
def renomear_arquivos(pasta, lista_nomes):
    lista_arquivos = [os.path.join(pasta, arquivo) for arquivo in os.listdir(pasta) if arquivo.endswith('.pdf')]
    lista_arquivos.sort(key=os.path.getctime)  # Ordenar do mais antigo para o mais recente
    
    lista_nomes.reverse()  # Inverter a lista de nomes

    print("Ordem dos arquivos (antigo -> recente):")
    for arquivo in lista_arquivos:
        print(os.path.basename(arquivo))

    print("Ordem dos nomes (último -> primeiro):")
    for nome in lista_nomes:
        print(nome)

    for i, caminho_antigo in enumerate(lista_arquivos):
        if i < len(lista_nomes):
            nome_cliente = lista_nomes[i]
            novo_nome = f'NFS-e - {nome_cliente}.pdf'
            caminho_novo = os.path.join(pasta, novo_nome)
            try:
                if os.path.isfile(caminho_antigo):  # Verificar se o arquivo existe
                    os.rename(caminho_antigo, caminho_novo)
                    print(f'Renomeando {os.path.basename(caminho_antigo)} para {novo_nome}')
                else:
                    print(f"Arquivo não encontrado: {caminho_antigo}")
            except Exception as e:
                print(f"Erro ao renomear {os.path.basename(caminho_antigo)}: {e}")
        else:
            print(f"Nome não disponível para o arquivo: {os.path.basename(caminho_antigo)}")

def mesclar_pdfs(pasta_entrada, arquivo_saida, lista_nomes):
    merger = PdfMerger()
    # Criar um dicionário para mapear o nome original para o caminho do arquivo renomeado
    nome_para_caminho = {}
    for nome in lista_nomes:
        nome_arquivo = f'NFS-e - {nome}.pdf'
        caminho_arquivo = os.path.join(pasta_entrada, nome_arquivo)
        if os.path.exists(caminho_arquivo):
            nome_para_caminho[nome] = caminho_arquivo

    # Adicionar os arquivos ao merger na ordem original da lista de nomes
    lista_nomes.reverse()  # Inverter a lista de nomes
    for nome in lista_nomes:
        caminho_arquivo = nome_para_caminho.get(nome)
        if caminho_arquivo:
            merger.append(caminho_arquivo)

    merger.write(arquivo_saida)
    merger.close()

# Função para executar o processo
def executar_processo():
    pasta = entrada_pasta.get()
    planilha = entrada_planilha.get()
    
    if not pasta or not planilha:
        messagebox.showerror("Erro", "Por favor, selecione a pasta e a planilha.")
        return

    lista_nomes = obter_lista_nomes(planilha)
    renomear_arquivos(pasta, lista_nomes)
    arquivo_saida = os.path.join(pasta, 'PDFao_Mescladao.pdf')
    mesclar_pdfs(pasta, arquivo_saida, lista_nomes)  # Passar a lista de nomes original

    messagebox.showinfo("Concluído", "Processo concluído com sucesso!")

# Configuração da janela principal
janela = tk.Tk()
janela.title("Gestor de Arquivos de NF - V 1.00")
janela.geometry("400x350")

# Carregar e exibir o logotipo
try:
    imagem_logo = Image.open("logo.png")
    largura_original, altura_original = imagem_logo.size
    nova_largura = int(largura_original * 0.25)
    nova_altura = int(altura_original * 0.25)
    imagem_logo = imagem_logo.resize((nova_largura, nova_altura), Image.LANCZOS)
    logo = ImageTk.PhotoImage(imagem_logo)
    label_logo = tk.Label(janela, image=logo)
    label_logo.place(relx=0.025, rely=0.025, anchor='nw')
except Exception as e:
    print(f"Erro ao carregar o logotipo: {e}")

# Adicionar título
titulo = tk.Label(janela, text="Renomeador de NFSe", font=("Calibri", 17, "bold"), fg="#555555")
titulo.place(relx=0.5, rely=0.03, anchor='n')

# Ajuste para posicionar os elementos abaixo da logo
offset_y = nova_altura + 50

# Entrada para a pasta
entrada_pasta = tk.StringVar()
tk.Label(janela, text="Selecione a pasta de PDFs:").pack(pady=(offset_y, 0))
tk.Entry(janela, textvariable=entrada_pasta, width=50).pack()
tk.Button(janela, text="Procurar", command=selecionar_pasta).pack(pady=5)

# Entrada para o arquivo Excel
entrada_planilha = tk.StringVar()
tk.Label(janela, text="Selecione a planilha Excel:").pack(pady=(40, 0))
tk.Entry(janela, textvariable=entrada_planilha, width=50).pack()
tk.Button(janela, text="Procurar", command=selecionar_planilha).pack(pady=5)

# Botão para executar o processo
tk.Button(janela, text="Executar", command=executar_processo, bg="#FF5733", fg="white").pack(pady=20)

janela.mainloop()